In [1]:
import pandas as pd
import spotipy
import numpy as np
import json
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
import re

In [2]:
with open('./secret.json','r') as f:
    secret = json.load(f)

In [3]:
client_id =secret['client_id']
client_secret = secret['client_secret']
username = secret['username']
redirect_uri="http://localhost:7777/callback"
scope = "user-library-read"

In [7]:
df = pd.read_csv("./Saved Data/Lyrics+Features.csv")

In [8]:
df=df.rename(columns={"Album":"track_album","Artist":"track_artist","Track":"track_title","TrackURI":"track_uri" , "TrackID":"track_id" ,"Lyrics":"track_lyric"})
df = df.drop(columns=["track_lyric"],axis=0)
df.head()

,track_album,track_artist,track_title,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,track_uri,track_id
0,Taylor Swift,Taylor Swift,Tim McGraw,0.580,0.491,0,-6.462,1,0.0251,0.575,0.0,0.1210,0.425,76.009,232107,4,spotify:track:0Om9WAB5RS09L80DyOfTNa,0Om9WAB5RS09L80DyOfTNa
1,Taylor Swift,Taylor Swift,Picture To Burn,0.658,0.877,7,-2.098,1,0.0323,0.173,0.0,0.0962,0.821,105.586,173067,4,spotify:track:32mVHdy0bi1XKgr0ajsBlG,32mVHdy0bi1XKgr0ajsBlG
2,Taylor Swift,Taylor Swift,Teardrops On My Guitar,0.621,0.417,10,-6.941,1,0.0231,0.288,0.0,0.1190,0.289,99.953,203040,4,spotify:track:7zMcNqs55Mxer82bvZFkpg,7zMcNqs55Mxer82bvZFkpg
3,Taylor Swift,Taylor Swift,A Place In This World,0.576,0.777,9,-2.881,1,0.0324,0.051,0.0,0.3200,0.428,115.028,199200,4,spotify:track:73OX8GdpOeGzKC6OvGSbsv,73OX8GdpOeGzKC6OvGSbsv
4,Taylor Swift,Taylor Swift,Cold as You,0.418,0.482,5,-5.769,1,0.0266,0.217,0.0,0.1230,0.261,175.558,239013,4,spotify:track:7an1exwMnfYRcdVQm0yDev,7an1exwMnfYRcdVQm0yDev


In [9]:
pattern=r"(\(.+\))"
for index,i in enumerate(df['track_title']):
    df['track_title'][index]=(re.sub(pattern,"",i).strip())

C:\Users\ishaa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
from gspread_pandas import Spread, Client
from gspread_pandas.conf import get_config
#get_config(conf_dir=None, file_name='google_secret.json')
google_secret = get_config(conf_dir="./", file_name='google_secret.json')

#spread = Spread("1WngV96GZrPp6WdmmBI2tmnKYEJETL7SGnivWIlPjFjM")
spread = Spread("1fSM-44NV0kzNmxkOB-90N3bKPrb2jiszQMgU_YNqagA")

#spread.df_to_sheet(finalData, index=False, sheet='Master', start='A1', replace=True)
df_gen=spread.sheet_to_df(index=0, header_rows=1, start_row=1, unformatted_columns=None, formula_columns=None, sheet='Genre')

In [21]:
df_gen

,Country,Country Pop,Country Rock,Contemporary Country,Blue grass,Pop,Folk Pop,Electropop,Synth Pop,Pop Rock,Dream Pop,Dance Pop,Soft Rock,Alternative Rock,Pop Punk,R&B
0,Tim McGraw,Teardrops On My Guitar,Picture To Burn,Tell Me Why,Mean,We Are Never Ever Getting Back Together,Speak Now,I Knew You Were Trouble,Welcome to New York,Change,Wildest Dreams,Shake It Off,Innocent,Haunted,Better Than Revenge,I Don't Wanna Live Forever
1,Cold as You,A Place In This World,Should've Said No,The Way I Loved You,,22,Never Grow Up,Starlight,Style,The Story of Us,This Love,Wonderland,All Too Well,The Last Time,Paper Rings,End Game
2,Our Song,The Outside,Youre Not Sorry,,,The Lucky One,Last Kiss,Blank Space,Out of the Woods,If This Was a Movie,Delicate,Look What You Made Me Do,Ronan,State of Grace,,Call It What You Want
3,I'd Lie,Tied Together With A Smile,Long Live,,,Come Back... Be Here,Stay Stay Stay,...Ready for It?,Getaway Car,Treacherous,The Archer,The Man,,Eyes Open,,So It Goes...
4,Permanent Marker,Stay Beautiful,Dear John,,,Sweeter Than Fiction,I Almost Do,Dancing With Our Hands Tied,Dress,Enchanted,Lover,Cornelia Street,,It's Nice to Have a Friend,,
5,All Over Me,Mary's Song,,,,Bad Blood,Everything Has Changed,This Is Why We Can't Have Nice Things,King of My Heart,Holy Ground,False God,,,,,
6,White Horse,I'm Only Me When I'm With You,,,,New Romantics,Sad Beautiful Tragic,I Did Something Bad,Don't Blame Me,Afterglow,,,,,,
7,The Best Day,Invisible,,,,All You Had to Do Was Stay,I Forgot That You Existed,I Think He Knows,London Boy,,,,,,,
8,I Heart ?,Fearless,,,,Superman,,Miss Americana & the Heartbreak Prince,Cruel Summer,,,,,,,
9,Ours,Fifteen,,,,I Wish You Would,,,The Man,,,,,,,


In [22]:
genredict={}
for column in df_gen.columns:
    #print(column)
    for track in df_gen[column]:
        if track in list(df['track_title']):
           genredict[track]=column 

In [23]:
genre_list=[]
for index,i in enumerate(df['track_title']):
    if i in genredict.keys():
        genre_list.append(genredict[i])
    else:
        genre_list.append(np.nan)

In [24]:
df['Genre'] = genre_list

In [25]:
df.loc[df['Genre'].isna(),'track_title']

Series([], Name: track_title, dtype: object)

In [26]:
len(df.loc[df['Genre'].isna(),'track_title'])

0

In [27]:
df_new = pd.read_csv("./Saved Data/Lyrics+Features.csv")

In [28]:
df_new['genres'] = df['Genre']
df_new=df_new.rename(columns={"Album":"track_album","Artist":"track_artist","Track":"track_title","TrackURI":"track_uri" , "TrackID":"track_id" ,"Lyrics":"track_lyric"})
df_new.to_csv("./Saved Data/Lyrics+Features+Genre.csv",index=False)

### Checking for all the songs whose entries dont exist

In [29]:
alltracks = []
for column in df_gen.columns:
    for elem in df_gen[column]:
        if elem is not '':
            alltracks.append(elem)

In [30]:
pattern=r"(\(.+\))"
for index,i in enumerate(df_new['track_title']):
    df_new['track_title'][index]=(re.sub(pattern,"",i).strip())

C:\Users\ishaa\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
for track in alltracks:
    if track not in list(df_new.track_title):
        print(track)

I'd Lie
Permanent Marker
All Over Me
I Heart ?
Today Was A Fairytale
Beautiful Eyes
Sweeter Than Fiction
Ronan
Eyes Open
I Don't Wanna Live Forever
